# Hàm chuẩn hóa văn bản tiếng Việt với 5 bước
- Chuẩn hóa unicode 
- Chuẩn hóa dấu câu tiếng Việt
- Tách từ tiếng Việt
- Chuyển chữ viết thường
- Chuẩn hóa câu

In [1]:
import unicodedata
import regex as re
from pyvi import ViTokenizer, ViPosTagger

bang_nguyen_am= [['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
                  ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
                  ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
                  ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e'],
                  ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
                  ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i'],
                  ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o'],
                  ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
                  ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
                  ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u'],
                  ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
                  ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']]

bang_ky_tu_dau = ['', 'f', 's', 'r', 'x', 'j']
nguyen_am_to_ids = {}

for i in range(len(bang_nguyen_am)):
    for j in range(len(bang_nguyen_am[i]) - 1):
        nguyen_am_to_ids[bang_nguyen_am[i][j]] = (i, j)

# Chuẩn hóa unicode 
# Có 2 loại unicode : unicode tổ hơp và unicode dựng sẵn, điêu này dẫn tới việc 2 từ giống nhau sẽ bị coi là khác nhau 
# Chuẩn hóa tất cả về 1 loại là unicode dựng sẵn
def chuan_hoa_unicode(text):
	text = unicodedata.normalize('NFC', text)
	return text

# Có 2 kiểu gõ dấu ở Tiếng Việt, ví dụ như là : òa hoặc oà (ta gọi lần lượt là chuẩn 1 và 2). Mặc dù kiểu gõ chữ sau ít 
#phổ biến hơn tuy nhiên vẫn cần phải chuẩn hóa tránh việc một số văn bản vẫn sử dụng kiểu gõ dấu thứ 2.
"""
	Hàm này xử lý chuẩn hóa từng từ một, sau khi chuẩn hóa từng từ thì ta sẽ đi chuân hóa từng câu sau 
	""" 
def chuan_hoa_dau_tu_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word
 
    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
        if not qu_or_gi or index != 1:
            nguyen_am_index.append(index)
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1])
                chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word
 
    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = bang_nguyen_am[x][dau_cau]
            # for index2 in nguyen_am_index:
            #     if index2 != index:
            #         x, y = nguyen_am_to_ids[chars[index]]
            #         chars[index2] = bang_nguyen_am[x][0]
            return ''.join(chars)
 
    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            # chars[nguyen_am_index[1]] = bang_nguyen_am[x][0]
        else:
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
        # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
        chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[2]]]
        # chars[nguyen_am_index[2]] = bang_nguyen_am[x][0]
    return ''.join(chars)

def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if index - nguyen_am_index != 1:
                    return False
                nguyen_am_index = index
    return True

def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
        Chuyển câu tiếng việt về chuẩn gõ dấu kiểu cũ.
        :param sentence:
        :return:
        """
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        # print(cw)
        if len(cw) == 3:
            cw[1] = chuan_hoa_dau_tu_tieng_viet(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)

# Tách từ tiếng việt, từ tiếng việt không giống như tiếng anh, tách từ tiếng anh ta chỉ cần tách bằng khoảng trắng
# Tuy nhiên từ tiếng Việt có cả từ đơn lẫn từ ghép nên tách từ tiêng Việt sẽ phúc tạp hơn 
# Project sử dung thu viện pyvi (xem mã nguồn tại : https://github.com/trungtv/pyvi) để phục vụ bài toán con tách từ Tiếng Việt 
def tach_tu_tieng_viet(text):
	text = ViTokenizer.tokenize(text)
	return text

# Đưa về chữ viết thường 
def chuyen_chu_thuong(text):
	return text.lower()

# Xóa đi các dấu cách thừa, các từ không cần thiết cho việc phân loại vẳn bản 
def chuan_hoa_cau(text):
	text = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',text)
	text = re.sub(r'\s+', ' ', text).strip()
	return text

def tien_xu_li(text):
	text = chuan_hoa_unicode(text)
	text = chuan_hoa_dau_cau_tieng_viet(text)
	text = tach_tu_tieng_viet(text)
	text = chuyen_chu_thuong(text)
	text = chuan_hoa_cau(text)

	return text

if __name__ == "__main__":
    file = open("test.txt", "r", encoding="utf-8")
    data = file.read()
    data = tien_xu_li(data)
    print("Ví dụ : \n")
    print(data)

Ví dụ : 

tp hcm phạt người không đeo khẩu_trang nơi công_cộng người dân ở thành_phố không đeo khẩu_trang nơi công_cộng sẽ bị xử_phạt mức cao nhất 300 000 đồng từ ngày 58 yêu_cầu này được chủ_tịch ubnd thành_phố nguyễn thành phong đưa ra tại cuộc họp ban chỉ_đạo phòng_chống dịch_bệnh covid 19 của tp hcm chiều 38 việc xử_phạt không đeo khẩu_trang nơi công_cộng được tp hcm cũng như các địa_phương khác thực_hiện từ cuối tháng 3 khi covid 19 bùng_phát tuy_nhiên sau khi hết thực_hiện cách_ly xã_hội từ ngày 234 việc đeo khẩu_trang nơi công_cộng chỉ dừng lại ở mức khuyến_cáo theo nghị_định số 1762013 người dân không đeo khẩu_trang nơi công_cộng sẽ bị xử_phạt từ 100 000 đến 300 000 đồng trong khoảng một tháng áp_dụng trước đó tp hcm đã xử_phạt hơn 4 300 trường_hợp với gần 870 triệu đồng theo ông phong việc đeo khẩu_trang đã được khẳng_định có_thể tránh lây_lan dịch_bệnh cho người khác và bảo_vệ sức_khỏe cho người sử_dụng sở công_thương phải nắm nguồn cung_ứng_khẩu_trang chủ_động thông_báo các 

# Chuẩn hóa toàn bộ tập dữ liệu

In [2]:
import os

# 27 nhãn
list_name_categories = os.listdir("raw-data/Test_Full")

def get_list_content(path):
	list_content_file = []
	list_name_file = os.listdir(path)
	for name_file in list_name_file : 
		# tạo đường dẫn của file từ thư muc cha và tên file 
		path_file = path + "/" + name_file
		# đọc file 
		file = open(path_file, "r", encoding="utf-16")
		# lấy nội dung của file 
		content_file = file.read()
		# chuẩn hóa nội dung file 
		content_file = tien_xu_li(content_file)
		# ghi file vào list 
		list_content_file.append(content_file)

	return list_content_file

def write_file_output(list_content_file, category, isTrain):
	path_file = ""
	if isTrain : 
		path_file = "clean-data/train/" + category + ".txt"
	else : 
		path_file = "clean-data/test/" + category + ".txt"

	file_output = open(path_file, "w", encoding="utf-8")
	for content_file in list_content_file : 
		file_output.write(content_file + "\n")

	print("write done " + category + " in path " + path_file + "!")
	file_output.close()

# chuẩn hóa tất cả các file của tất cả các nhãn 
def standardize_all_categories(train_or_test):
	for name_categories in list_name_categories : 
		path_file_output = "raw-data/" + train_or_test + "/" + name_categories
		list_content_file = get_list_content(path_file_output)	

		if train_or_test == "Train_Full":
			write_file_output(list_content_file, name_categories, True)
		else :
			write_file_output(list_content_file, name_categories, False)

if __name__ == "__main__":
	# main
	standardize_all_categories("Test_Full")
	standardize_all_categories("Train_Full")


write done Van hoa in path clean-data/test/Van hoa.txt!
write done Suc khoe in path clean-data/test/Suc khoe.txt!
write done Vi tinh in path clean-data/test/Vi tinh.txt!
write done Khoa hoc in path clean-data/test/Khoa hoc.txt!
write done Chinh tri Xa hoi in path clean-data/test/Chinh tri Xa hoi.txt!
write done The thao in path clean-data/test/The thao.txt!
write done Phap luat in path clean-data/test/Phap luat.txt!
write done Kinh doanh in path clean-data/test/Kinh doanh.txt!
write done Doi song in path clean-data/test/Doi song.txt!
write done The gioi in path clean-data/test/The gioi.txt!
write done Van hoa in path clean-data/train/Van hoa.txt!
write done Suc khoe in path clean-data/train/Suc khoe.txt!
write done Vi tinh in path clean-data/train/Vi tinh.txt!
write done Khoa hoc in path clean-data/train/Khoa hoc.txt!
write done Chinh tri Xa hoi in path clean-data/train/Chinh tri Xa hoi.txt!
write done The thao in path clean-data/train/The thao.txt!
write done Phap luat in path clean-d

# Chia lại tập train, test theo tỉ lệ của mình

In [3]:
# Module dùng để chia lại số lượng tập train và test
# Đồng thời gộp lại tập train và test để thành tập Full nếu cân 

import os

def data_aggregation():
	list_name = os.listdir("clean-data/train/")

	for name_file in list_name : 
		f_train = open("clean-data/train/" + name_file, "r", encoding="utf-8")
		f_test = open("clean-data/test/" + name_file, "r", encoding="utf-8")

		f_all = open("clean-data/full/" + name_file, "w", encoding="utf-8")

		data_f_train = f_train.read()
		data_f_test = f_test.read()

		f_all.write(data_f_train + "\n" + data_f_test)
		print("write done " + name_file)


def split_data(percent):
	list_name = os.listdir("clean-data/train/")

	for name_file in list_name : 
		f_all = open("clean-data/full/" + name_file, "r", encoding="utf-8")
		data_f_all = f_all.read()

		list_data_f_all = data_f_all.split("\n")
		size_data_f_all = len(list_data_f_all)

		size_data_f_train = int(size_data_f_all * 70 / 100) # lấy số lương văn bản cho tập train 
		f_train = open("clean-data/train/" + name_file, "w", encoding="utf-8")
		f_test = open("clean-data/test/" + name_file, "w", encoding="utf-8")

		cnt = 0
		for i_data_f_all in list_data_f_all : 
			if cnt < size_data_f_train : # nếu cnt < dung lượng tập train thì ghi vào tập train, không thì ghi vào tập test 
				if i_data_f_all != "" : 
					f_train.write(i_data_f_all + "\n")	
			else : # ghi ra tập test 
				if i_data_f_all != "" :
					f_test.write(i_data_f_all + "\n")

			cnt += 1

		print("[FULL] " + name_file + " : " + str(len(open("clean-data/full/"  + name_file, "r", encoding="utf-8").read().split("\n"))))
		print("[TRAIN] " + name_file + " : " + str(len(open("clean-data/train/" + name_file, "r", encoding="utf-8").read().split("\n"))))
		print("[TEST] " + name_file + " : " + str(len(open("clean-data/test/" + name_file, "r", encoding="utf-8").read().split("\n"))))

if __name__ == "__main__":
	# Nếu cần gộp data thì hãy bỏ comment để sử dụng hàm này 
	data_aggregation()
	split_data(70)

write done Doi song.txt
write done Kinh doanh.txt
write done Khoa hoc.txt
write done Phap luat.txt
write done Chinh tri Xa hoi.txt
write done The gioi.txt
write done Suc khoe.txt
write done The thao.txt
write done Vi tinh.txt
write done Van hoa.txt
[FULL] Doi song.txt : 5197
[TRAIN] Doi song.txt : 3637
[TEST] Doi song.txt : 1558
[FULL] Kinh doanh.txt : 7830
[TRAIN] Kinh doanh.txt : 5480
[TEST] Kinh doanh.txt : 2349
[FULL] Khoa hoc.txt : 3918
[TRAIN] Khoa hoc.txt : 2742
[TEST] Khoa hoc.txt : 1176
[FULL] Phap luat.txt : 7658
[TRAIN] Phap luat.txt : 5360
[TEST] Phap luat.txt : 2296
[FULL] Chinh tri Xa hoi.txt : 12788
[TRAIN] Chinh tri Xa hoi.txt : 8950
[TEST] Chinh tri Xa hoi.txt : 3837
[FULL] The gioi.txt : 9616
[TRAIN] The gioi.txt : 6731
[TEST] The gioi.txt : 2882
[FULL] Suc khoe.txt : 8803
[TRAIN] Suc khoe.txt : 6162
[TEST] Suc khoe.txt : 2640
[FULL] The thao.txt : 11967
[TRAIN] The thao.txt : 8376
[TEST] The thao.txt : 3589
[FULL] Vi tinh.txt : 7043
[TRAIN] Vi tinh.txt : 4930
[TEST] 

# Tính toán số lượng từ của từ điển

In [11]:
import os
import re

list_name_file = os.listdir("clean-data/train/")

# truy cập vào tập train và tạo từ điển cho tập train 
def get_dictionary(folder):
	for name_file in list_name_file : 
		path_file = folder + "/train/" + name_file
		file = open(path_file, "r", encoding="utf-8")
		data = file.read()

		list_words = re.split('[ \n]', data)
		for word in list_words : 
			if word not in dictionary : 
				dictionary[word] = 0
			else : 
				dictionary[word] += 1
		print("done! " + name_file)
	return dictionary

# lấy ra dung lượng của từ điển 
def size_of_dictionary(dictionary):
	cnt = 0
	for word in dictionary : 
		cnt += 1

	print("size of dictionary : " + str(cnt))

if __name__ == "__main__":
	# lấy từ điển 
	dictionary = get_dictionary("clean-data")

	# in số lượng từ điển 
	size_of_dictionary(dictionary)

done! Doi song.txt
done! Kinh doanh.txt
done! Khoa hoc.txt
done! Phap luat.txt
done! Chinh tri Xa hoi.txt
done! The gioi.txt
done! Suc khoe.txt
done! The thao.txt
done! Vi tinh.txt
done! Van hoa.txt
size of dictionary : 153368


# Xóa stopword và lower word trong từ điển
(Chỉ xóa stopword và lower word trong từ điên)

In [13]:
import os

stopword = []
lower_word = []

def remove_stopword(dictionary):
	new_dictionary = {}
	f_stopword = open("stopword.txt", "r", encoding="utf-8")
	data_stopword = f_stopword.read().split("\n")

	for data in data_stopword : 
		stopword.append(data)

	for word in dictionary : 
		if word not in stopword :
			new_dictionary[word] = dictionary.get(word)

	print("size of dictionary after remove stop word : " + str(len(new_dictionary)))
	return new_dictionary

def remove_lower_word(dictionary):
	new_dictionary = {}
	for word in dictionary : 
		if dictionary[word] < 30: 
			lower_word.append(word)
		else :
			new_dictionary[word] = dictionary.get(word)

	print("size of dictionary after remove lower word : " + str(len(new_dictionary)))
	return new_dictionary


# def remove_stopword_and_lowerword(dictionary):
	# list_name = os.listdir("clean-data/train/")
	# for name_file in list_name : 
	# 	# xử lý tập train trước 
	# 	data_f_train = open("clean-data/train/" + name_file, "r", encoding="utf-8").read()
	# 	data_train = open("data/train/" + name_file, "w", encoding="utf-8")
	# 	list_data_f_train = data_f_train.split("\n")

	# 	# xử lý từng văn bản trong mỗi nhãn 
	# 	for i_data_f_train in list_data_f_train : 
	# 		text = ""
	# 		list_i_data_f_train = i_data_f_train.split(" ")
	# 		for word in list_i_data_f_train : 
	# 			if word not in stopword and word not in lower_word : 
	# 				text += word + " "
	# 		data_train.write(text + "\n")
	# 	print("done! " + name_file)

def print_dictionary(dictionary):
	f = open("dictionary.txt", "w", encoding="utf-8")
	for word in dictionary :
		f.write(word + " " + str(dictionary.get(word)) + "\n")
	print(dictionary)
    
        
if __name__ == "__main__":
	dictionary = get_dictionary("clean-data")
	print("size of dictionary before remove stop word and lower word : " + str(len(dictionary)))

	dictionary = remove_stopword(dictionary)
	dictionary = remove_lower_word(dictionary)

	print_dictionary(dictionary)


done! Doi song.txt
done! Kinh doanh.txt
done! Khoa hoc.txt
done! Phap luat.txt
done! Chinh tri Xa hoi.txt
done! The gioi.txt
done! Suc khoe.txt
done! The thao.txt
done! Vi tinh.txt
done! Van hoa.txt
size of dictionary before remove stop word and lower word : 153368
size of dictionary after remove stop word : 153293
size of dictionary after remove lower word : 27533
{'em': 88499, 'quen': 11234, 'anh': 416649, 'lâu': 29624, 'đến': 540814, '6': 121599, '7': 84664, 'năm': 444729, 'ít': 56114, 'giờ_đây': 11234, 'nhỏ': 67134, 'bạn': 228504, 'trang_lứa': 414, 'hai': 269069, 'nhóc': 739, 'hỏi': 40534, 'chừng': 8329, 'nào': 139739, 'tổ_chức': 110324, 'bé': 36834, 'hồng': 37029, 'bưng': 354, 'quả': 34404, 'cô_dâu': 2819, 'nghe': 43899, 'ngập_ngừng': 259, 'cười': 18174, 'dại': 1634, 'cưới': 9184, 'thành': 92894, 'gái': 41349, 'mi': 5169, 'chắc': 16459, 'nghệ_thuật': 23719, 'cá_nhân': 29494, 'đóng': 34664, 'vai_trò': 19629, 'diễn_viên': 31914, 'vai': 35114, 'diễn': 81629, 'thành_công': 45429, 'nhà

# Chuyển đổi tập dữ liệu sang file csv

In [14]:
import pandas as pd
import numpy as np
import os 

def convert_txt_to_csv(train_or_test) :
    list_name_file = os.listdir("clean-data/" + train_or_test)
    cnt = 0
    for name_file in list_name_file : 
        path = "clean-data/" + train_or_test + "/" + name_file
        data = open(path)
        X =[]
        for x in data:
            x = x.replace("\n","")
            X.append(x)
        Y = np.zeros(len(X))
        data = pd.DataFrame(X,columns=['content'])
        data['lable'] = cnt
        print("table " + str(cnt) + " : \n")
        print(data)

        name_file_out = name_file.split(".")[0]
        path_out = "csv/" + train_or_test + "/" + name_file_out + ".csv"
        data.to_csv(path_out,index=False)
        cnt += 1
        
convert_txt_to_csv("train")
convert_txt_to_csv("test")

table 0 : 

                                                content  lable
0     em quen anh đã lâu có đến 6 7 năm gì đó chứ ít...      0
1     đó là nghệ_thuật mà mỗi cá_nhân đóng vai_trò d...      0
2     đối_với phần_lớn chúng_ta từ bí_mật có một ý_n...      0
3     hoàn_cảnh của tôi giống lài đọc những dòng tâm...      0
4     tạo không_gian mát_mẻ trong mùa hè để làm dịu ...      0
...                                                 ...    ...
3631  hoa_hậu nguyễn thu thủy để được nể_trọng rất k...      0
3632  làm gì để giúp trẻ vượt qua cảnh cha_mẹ ly_hôn...      0
3633  khi sao nữ dùng bạo_lực nhìn bề_ngoài họ dịu_d...      0
3634  chiến_lược chinh_phục mẹ chồng của jennifer ga...      0
3635  săn côn_trùng độc ở bảy núi những côn_trùng có...      0

[3636 rows x 2 columns]
table 1 : 

                                                content  lable
0     tiếp_tục giữ nguyên lãi_suất cơ_bản phó thống_...      1
1     cổ_phiếu ngân_hàng thu_hút các nhà đầu_tư là t...      1
2     

table 2 : 

                                                content  lable
0     đập marib phía nam ảrập hiếm có mưa trong vùng...      2
1     chùm ảnh ảnh_hưởng của đô_thị_hóa tới hành_tin...      2
2     phụ_nữ muốn gì sau đêm mặn_nồng không_thể nói ...      2
3     các nhà khoa_học tại viện công_nghệ georgia mỹ...      2
4     tái_tạo tượng phật khổng_lồ bị phá_hủy ở afgha...      2
...                                                 ...    ...
1170  bạn có hạnh_phúc nếu bạn có quá nhiều bí_mật c...      2
1171  cuộc_sống ven_đô thời la mã ở anh tắm suối nướ...      2
1172  mỹ chuẩn_bị khám_phá biên_giới hệ mặt_trời cơ_...      2
1173  bao cao_su chơi nhạc một nhà sáng_chế người uk...      2
1174  khinh khí_cầu thế_kỷ 21 mặc_dù có vẻ ngoài lạ_...      2

[1175 rows x 2 columns]
table 3 : 

                                                content  lable
0     dùng người_yêu làm mỹ_nhân_kế vợ đi xuất_khẩu ...      3
1     ba quyết_định của thành_phố hà_nội bị kiện tòa...      3
2     

# Hàm tính tf-idf cho từng văn bản với từ điển vừa được tạo bên trên

In [6]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer 

file = open("dictionary.txt",  encoding="utf8")
strs = file.read().split("\n")
strs=strs[:len(strs)-1]
keys=[]
for line in strs:
    s=line.split(" ")
    keys.append(s[0])
    
    
import os
path=os.getcwd()
path+="/clean-data/train"
paths=os.listdir(path)
strs =[]
for p in paths:
    file = open(path+"/"+p,  encoding="utf8")
    strTmp=file.read().split("\n")
    x=len(strTmp)
    strs+=strTmp[:x-1]


cv=CountVectorizer(vocabulary=keys)
word_count_vector=cv.fit_transform(strs)
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True) 
tfidf_transformer.fit(word_count_vector)

def computing_tfidf(s):
    input_s =[]
    input_s.append(s)
    count_vector=cv.transform(input_s) 
    tf_idf_vector=tfidf_transformer.transform(count_vector)
    values=tf_idf_vector.toarray().tolist()[0]
    dictionary = dict(zip(keys, values))
    return dictionary

In [7]:
text = "Sáng ngày 25/2, Nathan Lee tiếp tục khiến dân tình dậy sóng khi tung nhiều tin nhắn làm bằng chứng cho rằng Cao Thái Sơn chính là nhân vật đứng sau vụ bóc phốt sai lệch khách sạn vừa qua bằng cách nhờ người khác đăng thông tin sai lệch về số tài sản của anh. Còn nhớ cách đây không lâu, Nathan Lee bị 1 người bạn của Cao Thái Sơn tố nhận vơ Toà thị chính Paris là khách sạn của nhà và rao bán với mức giá 2500 tỷ đồng trong mùa dịch Covid-19."
text = tien_xu_li(text)
dic=computing_tfidf(text)
print(dic)

{'em': 0.0, 'quen': 0.0, 'anh': 0.054260077689125184, 'lâu': 0.08794303400565073, 'đến': 0.0, '6': 0.0, '7': 0.0, 'năm': 0.0, 'ít': 0.0, 'giờ_đây': 0.0, 'nhỏ': 0.0, 'bạn': 0.06805885352167987, 'trang_lứa': 0.0, 'hai': 0.0, 'nhóc': 0.0, 'hỏi': 0.0, 'chừng': 0.0, 'nào': 0.0, 'tổ_chức': 0.0, 'bé': 0.0, 'hồng': 0.0, 'bưng': 0.0, 'quả': 0.0, 'cô_dâu': 0.0, 'nghe': 0.0, 'ngập_ngừng': 0.0, 'cười': 0.0, 'dại': 0.0, 'cưới': 0.0, 'thành': 0.0, 'gái': 0.0, 'mi': 0.0, 'chắc': 0.0, 'nghệ_thuật': 0.0, 'cá_nhân': 0.0, 'đóng': 0.0, 'vai_trò': 0.0, 'diễn_viên': 0.0, 'vai': 0.0, 'diễn': 0.0, 'thành_công': 0.0, 'nhà': 0.04711969935979767, 'tâm_lý': 0.0, 'khuyên': 0.0, 'cặp': 0.0, 'vợ_chồng': 0.0, 'luôn': 0.0, 'đổi_mới': 0.0, 'sống': 0.0, 'tốt_đẹp': 0.0, 'kết_hợp': 0.0, 'hài_hòa': 0.0, 'nhuần_nhuyễn': 0.0, 'trong': 0.02835017194966084, 'suốt': 0.0, 'cuộc_đời': 0.0, 'một': 0.0, 'quy_tắc': 0.0, 'biết': 0.0, 'chính': 0.05985459694623536, 'mình': 0.0, 'cụ_thể': 0.0, 'thích': 0.0, 'làm': 0.04309576706166738, '